# Linear programming tools and demonstrations
By <a href="https://www.maxkapur.com/">Max Kapur</a>. Updated Feb. 9, 2020.

## Pivoting a system of linear equations in Sympy
Reference: Robert J. Vanderbei, *Linear Programming: Foundations and Extensions,* ch. 2.

Rewritten in matrix form, problem 2.3 has us maximize

$$\zeta =  2x_1 - 6x_2 $$

subject to 

$$\begin{align}
\begin{bmatrix} -1 & -1 & -1 \\
2 & -1 & 1\\ \end{bmatrix} x &\leq \begin{bmatrix} -2 \\ 1 \end{bmatrix} \\ \\
x &\geq 0\end{align}$$

(The vector $x$ is of length 3.)

Since both $c$ and $b$ contain a mix of negative values, the starting dictionary is neither primal nor dual feasible; we need to use a Phase I algorithm to generate a feasible starting dictionary. 

My purpose, however, is to simply use sympy to develop a general tool that allows us to pivot the dictionary and test various bases. 

In [54]:
# 2.3

import sympy as sp
sp.init_printing()

# Variables and slack variables
x1, x2, x3, w1, w2 = sp.symbols(" x1, x2, x3, w1, w2 ")
x = sp.Matrix([x1, x2, x3])
w = sp.Matrix([w1, w2])

# Data
c = sp.Matrix([2, -6,0])
A = sp.Matrix([[1, 1, 1],[-2, 1, -1]])*-1
b = sp.Matrix([-2, 1])

# Constraints
con = sp.Eq(w, b - (A*x))
con

⎡w₁⎤   ⎡ x₁ + x₂ + x₃ - 2  ⎤
⎢  ⎥ = ⎢                   ⎥
⎣w₂⎦   ⎣-2⋅x₁ + x₂ - x₃ + 1⎦

In [86]:
# Returns the above dictionary pivoted with the given x_i or w_j as the basis.
def pivot(basic):
    constraints = sp.solve(con, basic)
    return{'obj': (c.T * x).subs(constraints), # Objective function
           'con': constraints}                 # Pivoted constraint functions

I happen to know that the optimal solution has $x_2$ and $x_3$ basic, but in the simplex method we would swap variables into the basis one at a time according to the set of pivoting rules chosen.

In [81]:
pivot((x2, x3))['obj']

[-3⋅w₁ - 3⋅w₂ - x₁ - 3]

In [85]:
pivot((x2, x3))['con']

Note that the constant terms in $x_2$ and $x_3$ are positive, while the cost coefficients in the current dictionary&rsquo;s objective function are all negative. That implies that this dictionary is optimal, with $x = (0, 1/2, 3/2)$ and $\zeta = -3$.

## Upper and lower bounds for a worst-case simplex problem

Problem 9 asks, 
> Show that, for any integer *n,*
> $$\frac{1}{2n} 2^{2n} \leq \binom{2n}{n} \leq 2^{2n}$$

These are tighter bounds than those <a href="https://math.stackexchange.com/questions/1352338/proof-for-the-upper-bound-and-lower-bound-for-binomial-coefficients">commonly given for the binomial coefficient</a>, so the proof is worthwhile.
    
We have 

$$\begin{align}
\binom{2n}{n} = \frac{(2n)!}{n!\,n!} = \left(\frac{2n}{n}\right)\left(\frac{2n-1}{n}\right)\left(\frac{2n-2}{n-1}\right)\left(\frac{2n-3}{n-1} \right)\left(\frac{2n-4}{n-2}\right)\left(\frac{2n-5}{n-2}\right)\biggl(\frac{\quad\cdots\quad}{\cdots}\biggr)\left(\frac{2}{1}\right)\left(\frac{1}{1}\right)
\end{align}$$

On the right, where I have spliced together the $n!$s in the denominator, note that for the $n$ &ldquo;even&rdquo; terms, 

$$\begin{align}
\left\lbrace\frac{2n}{n},  \frac{2n-2}{n-1}, \frac{2n-4}{n-2},  \cdots,  \frac{2}{1}\right\rbrace = 2
\end{align}$$

while for the $n$ &ldquo;odd&rdquo; terms, 

$$\begin{align}
\left\lbrace\frac{2n-1}{n} ,  \frac{2n-3}{n-1}, \frac{2n-5}{n-2} , \cdots , \frac{1}{1}\right\rbrace  \leq 2
\end{align}$$

Thus, the entire expression has

$$\begin{align}
\binom{2n}{n} \leq (2)(2) \cdots (2)(2) = 2^{2n}
\end{align}$$

and the upper bound is established.

For the lower bound, it suffices to show that $ \binom{2n}{n}\left(2n\right) \geq  2^{2n} $. We have


$$\begin{align}
\binom{2n}{n}\left(2n\right) &= \frac{(2n)!}{n!\,n!} \left(2n\right) \\
&= \left(\frac{2n}{n}\right)\left(\frac{2n-1}{n}\right)\left(\frac{2n-2}{n-1}\right)\left(\frac{2n-3}{n-1} \right)\left(\frac{2n-4}{n-2}\right)\left(\frac{2n-5}{n-2}\right)\biggl(\frac{\quad\cdots\quad}{\cdots}\biggr)\left(\frac{2}{1}\right)\left(\frac{1}{1}\right) \left(2n\right)\\
&= \Biggl[\left(\frac{2n}{n}\right)\left(\frac{2n-2}{n-1}\right)\left(\frac{2n-4}{n-2}\right)\biggl(\frac{\quad\cdots\quad}{\cdots}\biggr)\left(\frac{2}{1}\right)\Biggr]
\Biggl[\left(\frac{2n-1}{n}\right)\left(\frac{2n-3}{n-1}\right)\left(\frac{2n-5}{n-2}\right)\biggl(\frac{\quad\cdots\quad}{\cdots}\biggr)\left(\frac{1}{1}\right)\Biggr]  \left(2n\right)\\
\end{align}$$

The first group equals $2^n$. Now pull $2n$ inside the second group and nudge each numerator over to the right. Then increase all but the first of the denominators in the second group by $1/2$ (which decreases the value of the expression) and factor $2$ out of the numerators:

$$\begin{align}
\binom{2n}{n}\left(2n\right) &= \Biggl[\;2^n\;\Biggr]
\Biggl[\left(\frac{2n}{n}\right)\left(\frac{2n-1}{n-1}\right)\left(\frac{2n-3}{n-2}\right)\biggl(\frac{\quad\cdots\quad}{\cdots}\biggr)\left(\frac{3}{1}\right)\Biggr] \\
&\geq \Biggl[\;2^n\;\Biggr]
\Biggl[\left(\frac{2n}{n}\right)\left(\frac{2\left(n-\frac{1}{2}\right)}{n-\frac{1}{2}}\right)\left(\frac{2\left(n-\frac{3}{2}\right)}{n-\frac{3}{2}}\right)\biggl(\frac{\quad\cdots\quad}{\cdots}\biggr)\left(\frac{3}{\frac{3}{2}}\right)\Biggr] = 2^n \cdot 2^n = 2^{2n} \\
\end{align}$$

This establishes the lower bound.

In [87]:
!jupyter nbconvert linprog.ipynb

[NbConvertApp] Converting notebook linprog.ipynb to html
[NbConvertApp] Writing 286624 bytes to linprog.html
